In [1]:
import pandas as pd
import nltk
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Embedding, Input,LSTM,GlobalMaxPool1D, Dropout
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
true_df = pd.read_csv("C:\\Users\\simra\\Downloads\\NLP Project\\News-_dataset\\True.csv")
fake_df=pd.read_csv("C:\\Users\\simra\\Downloads\\NLP Project\\News-_dataset\\Fake.csv")

In [5]:
#Add label Column
true_df['label'] = 0
fake_df['label'] = 1
#true_df

In [7]:
#Combined Both Files 
combined_df = pd.concat([true_df, fake_df])
#combined_df

In [9]:
combined_df=combined_df.drop(['date'],axis=1)


In [11]:
combined_df['Text'] = combined_df['title'] + " " + combined_df['text'] + " " + combined_df['subject']
combined_df=combined_df.drop(['title','text','subject'],axis=1)

In [13]:
y = combined_df['label'].values
X = combined_df.drop(['label'],axis=1)

In [15]:
# configuration settingof LSTM (According to Research Paper)
voc_size = 10000
sent_length = 1000
Test_size = 0.3
batch_size = 120
EPOCHS = 30 # original 164 but Progress is not shown
Learning_rate = 0.0001
embedding_vector_features = 100

In [17]:
onehot_repr = [one_hot(words, voc_size) for words in X['Text']]

In [19]:
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(embedded_docs, y, test_size=Test_size, random_state=42)

In [23]:
model = Sequential()

# Add layers to the model
model.add(Input(shape=(sent_length,)))
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
optimizer = Adam(learning_rate=Learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 1000, 100)           │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,080,501 (4.12 MB)

 Trainable params: 1,080,501 (4.12 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=batch_size)

Epoch 1/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 1029s 4s/step - accuracy: 0.7972 - loss: nan
Epoch 2/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 990s 4s/step - accuracy: 0.4744 - loss: nan
Epoch 3/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 1038s 4s/step - accuracy: 0.4785 - loss: nan
Epoch 4/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 628s 2s/step - accuracy: 0.4731 - loss: nan
Epoch 5/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 625s 2s/step - accuracy: 0.4756 - loss: nan
Epoch 6/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 629s 2s/step - accuracy: 0.4767 - loss: nan
Epoch 7/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 630s 2s/step - accuracy: 0.4773 - loss: nan
Epoch 8/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 623s 2s/step - accuracy: 0.4781 - loss: nan
Epoch 9/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 623s 2s/step - accuracy: 0.4735 - loss: nan
Epoch 10/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 616s 2s/step - accuracy: 0.4759 - loss: nan
Epoch 11/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 615s 2s/step - accuracy: 0.4718 - loss: nan
Epoch 12/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 607s 2s/step - accuracy: 0.4785

In [27]:
y_pred = model.predict(X_test)

421/421 ━━━━━━━━━━━━━━━━━━━━ 108s 254ms/step


In [29]:
y_pred[:5]

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [31]:
#print(classification_report(y_test,y_pred.round()))

C:\Users\simra\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:290: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_pred contains NaN.